In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier


In [6]:
#data_path = r"E:\2nd year\Winter project\Thyroid Performance Analysis\Data"

In [7]:
 data_path = r"C:\Users\dwija\OneDrive\Thyroid-Performance-Analysis\Data"

In [8]:
ann_test = pd.read_csv(data_path+r'\anntest.CSV')
ann_train = pd.read_csv(data_path+r'\anntrain.CSV')
hyper_test = pd.read_csv(data_path+r'\hyperTest.CSV')
hyper_train = pd.read_csv(data_path+r'\hyperTrain.CSV')
hypo_test = pd.read_csv(data_path+r'\hypoTest.CSV')
hypo_train = pd.read_csv(data_path+r'\hypoTrain.CSV')
all_hypo_train = pd.read_csv(data_path+r'\hypothyroid.csv')
euthyroid = pd.read_csv(data_path+r'\sick-euthyroid.CSV')
thyroid0387 = pd.read_csv(data_path+r'\thyroid0387EDIT.CSV')

In [9]:
data = {
    'ann test': ann_test,
    'ann train': ann_train,
    'hyper test': hyper_test,
    'hyper train': hyper_train,
    'hypo test': hypo_test,
    'hypo train': hypo_train,
}


In [10]:
hypo_hyper_data ={    'hyper test': hyper_test,
    'hyper train': hyper_train,
    'hypo test': hypo_test,
    'hypo train': hypo_train,
}

In [11]:
ann_train = ann_train.drop_duplicates()
ann_test = ann_test.drop_duplicates()

In [12]:
for df_name, df in hypo_hyper_data.items():
        df.drop("ID", axis=1, inplace=True)
        df.drop("referral_source", axis=1, inplace=True)

In [13]:
hyper_test['Target'] = hyper_test['Target'].replace(["hyperthyroid", "T3_toxic", "goitre", "secondary_toxic"], "hyperthyroid")
hyper_train['Target'] = hyper_train['Target'].replace(["hyperthyroid", "T3_toxic", "goitre", "secondary_toxic"], "hyperthyroid")
hypo_test['Target'] = hypo_test['Target'].replace(["hypothyroid", "primary_hypothyroid", "compensated_hypothyroid", "secondary_hypothyroid"],"hypothyroid")
hypo_train['Target'] = hypo_train['Target'].replace(["hypothyroid", "primary_hypothyroid", "compensated_hypothyroid", "secondary_hypothyroid"],"hypothyroid")

In [14]:
Dataset = pd.concat([hyper_test,hyper_train,hypo_test,hypo_train], ignore_index = True)

In [15]:
Dataset['sex'] = Dataset['sex'].replace({'M': 0, 'F': 1})

In [16]:
Dataset.replace('?', np.nan, inplace=True)

In [17]:
Dataset = Dataset.drop(['TBG', 'TBG_measured','sex'], axis=1)

In [18]:
Dataset.dropna(axis = 0, thresh = 22, inplace = True)
Dataset.isna().sum()

age                             2
on_thyroxine                    0
query_on_thyroxine              0
on_antithyroid_medication       0
sick                            0
pregnant                        0
thyroid_surgery                 0
I131_treatment                  0
query_hypothyroid               0
query_hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                   0
psych                           0
TSH_measured                    0
TSH                           380
T3_measured                     0
T3                           1180
TT4_measured                    0
TT4                           104
T4U_measured                    0
T4U                           416
FTI_measured                    0
FTI                           412
Target                          0
dtype: int64

In [19]:
Dataset = Dataset.replace({"t":1,"f":0, "y":1, "n":0, "hypothyroid":1, "negative":0,"hyperthyroid":2, "F":1, "M":0})
display(Dataset.dtypes)

age                          object
on_thyroxine                  int64
query_on_thyroxine            int64
on_antithyroid_medication     int64
sick                          int64
pregnant                      int64
thyroid_surgery               int64
I131_treatment                int64
query_hypothyroid             int64
query_hyperthyroid            int64
lithium                       int64
goitre                        int64
tumor                         int64
hypopituitary                 int64
psych                         int64
TSH_measured                  int64
TSH                          object
T3_measured                   int64
T3                           object
TT4_measured                  int64
TT4                          object
T4U_measured                  int64
T4U                          object
FTI_measured                  int64
FTI                          object
Target                        int64
dtype: object

In [20]:
cols = Dataset.columns[Dataset.dtypes.eq('object')]
Dataset[cols] = Dataset[cols].apply(pd.to_numeric, errors='coerce')
display(Dataset.dtypes)

age                          float64
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment                 int64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                  int64
psych                          int64
TSH_measured                   int64
TSH                          float64
T3_measured                    int64
T3                           float64
TT4_measured                   int64
TT4                          float64
T4U_measured                   int64
T4U                          float64
FTI_measured                   int64
FTI                          float64
Target                         int64
dtype: object

In [21]:
Dataset = Dataset.interpolate(method = 'spline', order = 3)

In [22]:
Dataset.isna().sum()

age                          0
on_thyroxine                 0
query_on_thyroxine           0
on_antithyroid_medication    0
sick                         0
pregnant                     0
thyroid_surgery              0
I131_treatment               0
query_hypothyroid            0
query_hyperthyroid           0
lithium                      0
goitre                       0
tumor                        0
hypopituitary                0
psych                        0
TSH_measured                 0
TSH                          0
T3_measured                  0
T3                           0
TT4_measured                 0
TT4                          0
T4U_measured                 0
T4U                          0
FTI_measured                 0
FTI                          0
Target                       0
dtype: int64

In [23]:
Dataset = Dataset.dropna()

In [42]:
Dataset.to_csv(r"C:\Users\dwija\OneDrive\Thyroid-Performance-Analysis\Data\pre_data\Dataset.csv")

In [24]:
X = Dataset.drop('Target', axis=1)  
y = Dataset['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
def results(y_test,y_pred):
    precision = precision_score(y_test, y_pred, average='weighted',zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1-Score: {f1 * 100:.2f}%")
    print(f"Accuracy: {accuracy * 100:.2f}%")

In [26]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
results(y_test,y_pred_rf)

Precision: 90.67%
Recall: 90.47%
F1-Score: 90.57%
Accuracy: 90.47%


In [27]:
svm_model = SVC(kernel='linear')  # You can choose different kernels like 'linear', 'rbf', etc.
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
results(y_test,y_pred_svm)

Precision: 90.10%
Recall: 94.92%
F1-Score: 92.45%
Accuracy: 94.92%


In [28]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
X_test_array = X_test.values  
y_pred_knn = knn_model.predict(X_test_array) 
results(y_test, y_pred_knn)


c:\Users\dwija\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


Precision: 90.93%
Recall: 93.74%
F1-Score: 92.17%
Accuracy: 93.74%


In [29]:
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
results(y_test, y_pred_gb)

Precision: 95.95%
Recall: 89.01%
F1-Score: 91.47%
Accuracy: 89.01%


In [33]:
xgb_model = XGBClassifier(n_estimators=100, random_state=42)  
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

results(y_test, y_pred_xgb)

Precision: 91.99%
Recall: 91.66%
F1-Score: 91.82%
Accuracy: 91.66%


In [34]:
nb_model = BernoulliNB()
nb_model.fit(X_train, y_train)
y_pred_gnb = nb_model.predict(X_test)
results(y_test, y_pred_gnb)

Precision: 93.90%
Recall: 94.58%
F1-Score: 92.41%
Accuracy: 94.58%


In [35]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
results(y_test, y_pred_dt)

Precision: 91.21%
Recall: 90.89%
F1-Score: 91.05%
Accuracy: 90.89%


In [36]:
models_dict = {
    'RandomForest': rf_model,
    'SVC': svm_model,
    'KNeighbors': knn_model,
    'GradientBoosting': gb_model,
    'XGBoost': xgb_model,
    'BernoulliNB': nb_model,
    'DecisionTree': dt_model,
}

In [39]:
folder_path = r'C:\Users\dwija\OneDrive\Thyroid-Performance-Analysis\SAVED_MODEL'
file_name = 'models.pkl'
full_path = f'{folder_path}/{file_name}'

# Save the dictionary of models using pickle
with open(full_path, 'wb') as file:
    pickle.dump(models_dict, file)

print(f'Models saved to {full_path}')

Models saved to C:\Users\dwija\OneDrive\Thyroid-Performance-Analysis\SAVED_MODEL/models.pkl
